In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets

In [44]:
# MNIST dataset
train_dataset = datasets.MNIST(root='./data',
                                train=True,
                                transform=transforms.ToTensor(),
                                download=True)

test_dataset = datasets.MNIST(root='./data',
                                train=False,
                                transform=transforms.ToTensor())

In [45]:
# Training the model
n_epochs =  5 # Number of epochs
batch_size = 128
learning_rate = 0.1

In [46]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [47]:
# Define the CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # First convolutional layer
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        # Second convolutional layer
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        # Fully connected layer
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        
        x = x.view(-1, 64 * 7 * 7)  # Flatten the tensor
        
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        
        return x  # Return raw logits

In [48]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [49]:
for epoch in range(n_epochs):
    model.train()
    for batch_idx, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update the weights

        # Print loss for each iteration
        if batch_idx%100 == 0:
            print(f'Epoch [{epoch+1}/{n_epochs}], Iteration [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    print(f'Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.4f}')

Epoch [1/5], Iteration [1/469], Loss: 2.2952
Epoch [1/5], Iteration [101/469], Loss: 0.4495
Epoch [1/5], Iteration [201/469], Loss: 0.2513
Epoch [1/5], Iteration [301/469], Loss: 0.1945
Epoch [1/5], Iteration [401/469], Loss: 0.1637
Epoch 1/5, Loss: 0.1245
Epoch [2/5], Iteration [1/469], Loss: 0.0417
Epoch [2/5], Iteration [101/469], Loss: 0.1723
Epoch [2/5], Iteration [201/469], Loss: 0.0811
Epoch [2/5], Iteration [301/469], Loss: 0.0959
Epoch [2/5], Iteration [401/469], Loss: 0.0384
Epoch 2/5, Loss: 0.0377
Epoch [3/5], Iteration [1/469], Loss: 0.0571
Epoch [3/5], Iteration [101/469], Loss: 0.1287
Epoch [3/5], Iteration [201/469], Loss: 0.1012
Epoch [3/5], Iteration [301/469], Loss: 0.0380
Epoch [3/5], Iteration [401/469], Loss: 0.0523
Epoch 3/5, Loss: 0.0342
Epoch [4/5], Iteration [1/469], Loss: 0.0284
Epoch [4/5], Iteration [101/469], Loss: 0.0215
Epoch [4/5], Iteration [201/469], Loss: 0.0233
Epoch [4/5], Iteration [301/469], Loss: 0.0306
Epoch [4/5], Iteration [401/469], Loss: 0.0

In [50]:
# Evaluate the model
model.eval()
correct = 0
total = 0

In [51]:
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the 10000 test images: {100 * correct / total} %')

Accuracy of the model on the 10000 test images: 98.82 %
